In [1]:
import os
import zipfile
from pathlib import Path
import numpy as np
import pandas as pd

def is_kaggle():
    """Check if running in a Kaggle environment."""
    return "KAGGLE_KERNEL_RUN_TYPE" in os.environ

# Set Paths Based on Environment
if is_kaggle():
    print("Running in Kaggle environment...")

    # Kaggle-specific paths
    INPUT_PATH = Path("/kaggle/input/rts-phase-one-final")
    OUTPUT_PATH = Path("/kaggle/output/")
    WORKING_PATH = Path("/kaggle/working")
    TEMP_PATH = Path("/kaggle/temp/")

    # Import secrets
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("OPENAI_API_KEY")

else:
    print("Running in Local environment...")

    # Local paths
    BASE_DIR = Path("/Users/vishwanathn/Work/AI-Agent-Hack/code/rts_phase_one")
    INPUT_PATH = BASE_DIR / "data"
    OUTPUT_PATH = BASE_DIR / "output"
    WORKING_PATH = BASE_DIR / "working"
    TEMP_PATH = BASE_DIR / "temp"

    # Ensure directories exist locally
    for path in [INPUT_PATH, OUTPUT_PATH, WORKING_PATH, TEMP_PATH]:
        path.mkdir(parents=True, exist_ok=True)

    # Ensure Kaggle CLI is installed locally
    try:
        import kaggle
    except ImportError:
        print("Installing kaggle CLI...")
        os.system("pip install kaggle")

    # Check for Kaggle API Key
    kaggle_config_path = os.path.expanduser("~/.kaggle/kaggle.json")
    if not os.path.exists(kaggle_config_path):
        print(f"⚠️ Kaggle API key not found at {kaggle_config_path}. Please download kaggle.json and place it there.")
    else:
        os.chmod(kaggle_config_path, 0o600)  # Secure the API key file

    # Define datasets to download
    datasets = {
        "ready-to-ship-warehouse-replenishment": "ready-to-ship-warehouse-replenishment",
        "vishwanathsahaj/rts-phase-one": "rts-phase-one",
        "vishwanathsahaj/rts-phase-one-final": "rts-phase-one-final",
    }

    for dataset, folder_name in datasets.items():
        dataset_path = INPUT_PATH / folder_name
        zip_path = INPUT_PATH / f"{folder_name}.zip"  # Path to store ZIP file

        # Download only if ZIP doesn't exist
        if not zip_path.exists():
            print(f"📥 Downloading {dataset}...")
            os.system(f"kaggle datasets download -d {dataset} -p {INPUT_PATH}")
        
        # Check if ZIP was downloaded
        if not zip_path.exists():
            print(f"❌ Error: ZIP file {zip_path} was not found after download. Skipping extraction.")
            continue

        # Extract ZIP if dataset folder is missing
        if not dataset_path.exists():
            print(f"📂 Extracting {zip_path} into {dataset_path}...")
            dataset_path.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
            try:
                with zipfile.ZipFile(zip_path, "r") as zip_ref:
                    zip_ref.extractall(dataset_path)
            except zipfile.BadZipFile:
                print(f"❌ Error: {zip_path} is not a valid ZIP file.")
                continue

    print("\n✅ Data download and extraction complete.")

    from dotenv import load_dotenv
    load_dotenv()  # Load secrets from .env file in local environment
    secret_value = os.getenv("OPENAI_API_KEY")

# Print paths for reference
print(f"📂 INPUT_PATH: {INPUT_PATH}")
print(f"📂 OUTPUT_PATH: {OUTPUT_PATH}")
print(f"📂 WORKING_PATH: {WORKING_PATH}")
print(f"📂 TEMP_PATH: {TEMP_PATH}")



Running in Kaggle environment...
📂 INPUT_PATH: /kaggle/input/rts-phase-one-final
📂 OUTPUT_PATH: /kaggle/output
📂 WORKING_PATH: /kaggle/working
📂 TEMP_PATH: /kaggle/temp


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install crewai==0.86.0 crewai-tools==0.17.0 'crewai[tools]' -qq --use-deprecated=legacy-resolver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 76.0 MB/s eta 0:00:00
  Preparing metadata (

In [4]:
# Change Input Path
if is_kaggle():
    INPUT_PATH = INPUT_PATH
else:
    INPUT_PATH = INPUT_PATH / "rts-phase-one-final"
print(INPUT_PATH)

/kaggle/input/rts-phase-one-final


In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Git Check
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk(INPUT_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rts-phase-one-final/products.csv
/kaggle/input/rts-phase-one-final/inventory.csv
/kaggle/input/rts-phase-one-final/warehouse.csv
/kaggle/input/rts-phase-one-final/orders.csv
/kaggle/input/rts-phase-one-final/sales_report_summary.json


In [6]:
import os

# Set the environment variable
os.environ["OPENAI_API_KEY"] = secret_value

# Print confirmation (Avoid printing actual secrets)
print("Secret Loaded Successfully" if secret_value else "Secret Not Found")


Secret Loaded Successfully


In [7]:
from typing import List, Tuple, Dict
from pathlib import Path

from crewai_tools import BaseTool
from crewai import Agent, Crew, Process, Task

In [8]:
FULFILLMENT_DATA_DIR = WORKING_PATH / "fullfilment"
PRODUCT_DATA_FILE = INPUT_PATH / "products.csv"

## Tools

### Historic Sales Report Summary

In [9]:
# Historic Sales Report Summary

import json

REPORT_DATA_FILE = INPUT_PATH / "sales_report_summary.json"

ERROR_FILE_NOT_FOUND = "Sales report summary information is not available."
ERROR_YEAR_MONTH_NOT_FOUND = "No sales summary information found for the given year and month."


class HistoricSalesReportSummaryRetriever(BaseTool):
    name: str = "Historic Sales Report Summary Retriever"
    description: str = (
        "This tool retrieves the historic sales report from the database and returns it as a dataframe.",
        "The report contains the sales report summary of 'Ready To Ship' order of October, November and December months for the past 5 years.",
        "The summary report talks about products that sold like a hot cake and products that didn't sell at all.",
        "Tool takes year and month (Oct, Nov, Dec) as input and returns the summary report as text.",
    )
    _sales_report_summary = {}

    def _run(self, year: int, month: str) -> str:
        if not self._sales_report_summary:
            try:
                with open(REPORT_DATA_FILE) as report:
                    self._sales_report_summary = json.load(report)
            except FileNotFoundError:
                return ERROR_FILE_NOT_FOUND
        sales_summary = self._sales_report_summary.get(str(year), {}).get(month, {})
        if not sales_summary:
            return ERROR_YEAR_MONTH_NOT_FOUND
        return sales_summary

### Top Selling Products



Create temp directory to log fulfillment data

In [10]:
!mkdir -p /kaggle/temp/fulfillment
!ls /kaggle/temp

fulfillment


In [11]:
# Top Selling Products

ERROR_NO_PRODUCTS_SOLD = "No products sold in the given warehouse for the given period."

class TopSellingProductsByWarehouseForPreviousNDays(BaseTool):
    name: str = "Top selling Products by Warehouse for previous N number of days"
    description: str = (
        "Get the top selling products for a given warehouse for the previous N number of days.",
        "Takes warehouse id and number of days as input and returns the top selling products.",
        "Products are sorted based on the number of units sold.",
    )
    _products_details = None

    def get_fulfillment_data(self, num_days: int) -> pd.DataFrame:
        "List the csv files in the fulfillment directory sort the file names in decending order and return the top N files."
        fulfillment_files = sorted(FULFILLMENT_DATA_DIR.iterdir(), reverse=True)
        top_files = fulfillment_files[:num_days]
        return pd.concat([pd.read_csv(file) for file in top_files])

    def get_product_details(self, product_id: str) -> str:
        # extend the returned message as per need
        if not self._products_details:
            # read and load the products details file on first call
            self._products_details = pd.read_csv(PRODUCT_DATA_FILE)
        product = self._products_details[self._products_details.product_id == product_id]
        return product.name

    def prepare_message(self, num_days: int, top_products: List[str]) -> str:
        # extend the returned message as per need
        product_details = f"Following are the top selling products for the given warehouse for the previous {num_days} days:\n"
        for product_id in top_products:
            product_details += f"{self.get_product_details(product_id)}\n"
        return product_details

    def _run(self, warehouse_id: str, num_days: int) -> str:
        fulfillment_data = self.get_fulfillment_data(num_days)
        warehouse_sales = fulfillment_data[fulfillment_data["warehouse_id"] == warehouse_id]
        if warehouse_sales.empty:
            return ERROR_NO_PRODUCTS_SOLD
        top_selling_products = warehouse_sales.groupby("product_id")["quantity"].sum().sort_values(ascending=False).index.to_list()
        return self.prepare_message(num_days, top_selling_products)

### Warehouse Surge Capacity Increase Requester

In [12]:
# Warehouse Surge Capacity Increase Requester

from random import randint

class WarehouseSurgeCapacityIncreaseRequester(BaseTool):
    name: str = "Warehouse Surge Value Increase Requester"
    description: str = (
        "The system facilitates sending a request to the Finance team to increase the maximum stock holding value of a warehouse. It requires a recommended surge value as input, which represents a percentage increase in the warehouse's maximum stock capacity. This surge value must fall within the range of 0% to 100%.",
        "Once the input is provided, the system sends the request to the Finance team for review. The Finance team evaluates the request based on the company’s current financial status and decides whether to approve or reject the proposed increase.",
        "The tool outputs an integer value representing the recommended surge percentage, ensuring it is always within the valid range of 0 to 100.",
    )

    def _run(self, surge_percent: int) -> str:
        # sample implementation
        return randint(0, surge_percent)

### Trending Product by Warehouse

In [13]:
# Trending Product by Warehouse
ERROR_NO_TRENDS = "No trending products found for the given warehouse."


class TrendingProductByWarehouse(BaseTool):
    name: str = "Trending Product by Warehouse"
    description: str = (
        "Get the trending product for a given warehouse.",
        "Takes warehouse id as input and returns the trending product.",
        "Trending product is the product with continuous increase in sales for the last 3 number of days.",
    )
    _NUM_DAYS = 3
    _products_details = None

    def get_fullfilment_data(self, num_days: int) -> pd.DataFrame:
        "List the csv files in the fullfilment directory sort the file names in decending order and return the top N files."
        fullfilment_files = sorted(FULFILMENT_DATA_DIR.iterdir(), reverse=True)
        # assume each fullfilment file is for a day
        last_n_days = fullfilment_files[:num_days]
        return pd.concat([pd.read_csv(file) for file in last_n_days])

    def monotonically_increasing(array: List[int], period:int=3) -> bool:
        # https://en.wikipedia.org/wiki/Monotonic_function
        # A function is monotonically increasing if for all x and y
        # Here we are checking if the sales of the product is increasing for the last 3 days.
        if len(array) < period+1:
            return False
        sub_array = array[-(period+1):]
        return all( x < y for x, y in zip(sub_array, sub_array[1:]))

    def get_product_details(self, product_id: str) -> str:
        # extend the returned message as per need
        if not self._products_details:
            # read and load the products details file on first call
            self._products_details = pd.read_csv(PRODUCT_DATA_FILE)
        product = self._products_details[self._products_details.product_id == product_id]
        return product.name

    def _run(self, warehouse_id: str) -> str:
        fullfilment_data = self.get_fullfilment_data(self._NUM_DAYS)
        warehouse_sales = fullfilment_data[fullfilment_data["warehouse_id"] == warehouse_id]
        product_sales = warehouse_sales.groupby(["product_id", "date"])["quantity"].sum().reset_index()
        trending_products = product_sales.groupby("product_id")["quantity"].apply(list).apply(lambda x: self.monotonically_increasing(x)).to_frame()
        trending_products = trending_products[trending_products["quantity"]].index.to_list()
        if not trending_products:
            return ERROR_NO_TRENDS
        # change the message as per need to return one or more trending products
        return f"Trending product for warehouse {warehouse_id} is {self.get_product_details(trending_products[0])}."

# **Custom Tools**

# Product info finder
This tool gets information about a product

In [14]:
# Product info finder

# Tool code
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class ProductInfoFinderInput(BaseModel):
    product_id: str = Field(..., description="id of the product to find")
    products_file_path: str = Field(..., description="File path of products")


class ProductInfoHelper(BaseTool):
    name: str = "Product Info Helper"
    description: str = """When invoked with {product_id}, and path of products csv {products_file_path}
    ,it will return information about the product"""
    args_schema: Type[BaseModel] = ProductInfoFinderInput

    def _run(self,product_id: str , products_file_path: str) -> str:
        result = find_product(product_id, products_file_path
        )
        return result


# Helper Code
def find_product(product_id, csv_path):
    """
    Find product information from a CSV file based on product ID.

    Parameters:
    product_id (str): The ID of the product to search for
    csv_path (str): Path to the CSV file

    Returns:
    dict: Product information if found, None if not found
    """
    try:
        # Read the CSV file
        df = pd.read_csv(csv_path)

        # Find the product by ID
        product = df[df['product_id'] == product_id]

        # If product not found, return None
        if product.empty:
            return None

        # Convert the row to a dictionary
        product_info = {
            'product_id': product['product_id'].iloc[0],
            'name': product['name'].iloc[0],
            'category': product['category'].iloc[0],
            'sub_category': product['sub_category'].iloc[0],
            'description': product['description'].iloc[0]
        }

        return product_info

    except FileNotFoundError:
        print(f"Error: File {csv_path} not found")
        return None
    except Exception as e:
        print(f"Error: An unexpected error occurred - {str(e)}")
        return None

# Top Selling Categories
This tool gets top `n` categories

In [15]:
# Top n categories finder
from typing import Type
from crewai.tools import BaseTool
from pydantic import BaseModel, Field

class ProductCategoryAnalyzerInput(BaseModel):
    number_of_categories: int = Field(..., description="Number of categories to return.")
    orders_file: str = Field(..., description="File path of orders")
    products_file: str = Field(..., description="File path of products")


class ProductCategoryAnalyzer(BaseTool):
    name: str = "Category Analyzer"
    description: str = """When invoked with {n}, {orders_file} and {products_file},
    it will return the top selling {n} categories"""
    args_schema: Type[BaseModel] = ProductCategoryAnalyzerInput

    def _run(self, number_of_categories: int, orders_file: str , products_file: str) -> str:
        result = get_top_selling_categories(orders_file, products_file, number_of_categories)
        return result


# Helper Code
def get_top_selling_categories(orders_file: str, products_file: str, n: int = 5) -> List[Tuple[str, int]]:
    """Get top n selling categories by quantity."""
    # Load data from CSV files
    orders_df = pd.read_csv(orders_file)
    products_df = pd.read_csv(products_file)

    # Merge orders and products data on 'product_id'
    merged_df = orders_df.merge(products_df, on='product_id', how='left')

    # Group by 'category' and calculate total quantity sold
    category_sales = merged_df.groupby('category')['quantity'].sum()

    # Sort categories by quantity and get the top n
    top_categories = category_sales.sort_values(ascending=False).head(n)

    return list(zip(top_categories.index, top_categories.values))

# Utility Code

In [16]:
# Custom Exceptions

class ReadyToShipError(Exception):
    MESSAGE_TEMPLATE = ""

    def __str__(self):
        return self.MESSAGE_TEMPLATE.format(*self.message_args)

    def __repr__(self):
        return str(self)


class OutOfStockError(ReadyToShipError):
    MESSAGE_TEMPLATE = "Out of stock: {}"

    def __init__(self, product_id: str):
        self.message_args = (product_id,)

# Agent

In [17]:
from pydantic import BaseModel, Field

class Quantity(BaseModel):
    quantity: int = Field(description="The quantity of the product to order")

In [18]:
# Inventory Replenishment Service (Agent)

class InventoryReplenishmentService:
    def __init__(self, tools: List[object]):
        """
        Initialize the agent with the tools required to replenish the inventory
        """
        # AGENT ( WAREHOUSE INVENTORY PLANNER )
        self.agent = Agent(
            role="Warehouse Inventory Planner",
            goal="Ensure optimal inventory levels in the warehouse by managing product stock, forecasting demand, and placing timely orders to replenish inventory based on daily usage trends. ",
            backstory="You are an expert in inventory management specializing in e-commerce fulfillment."
            "Your skillset includes managing warehouse inventory and predicting order fluctuations to decide how much inventory should be stocked of each product in the warehouse. "
            "You will communicate your plans to replenish inventory with the finance and marketing teams each day to ensure approvals",
            allow_delegation=False,
        	verbose=True,
        )

        # TASK FOR THE AGENT
        self.task = Task(
            description=(
            "Given that {sold_quantity} units of product {product_id} were sold today, and {remaining_quantity} units are remaining, choose the "
            "quantity to order so that sufficient quantity is available for tomorrow's operations in the warehouse"
            ),
            output_pydantic=Quantity,
            expected_output="The quantity of the product to order, as a single number",
            agent=self.agent,
        )


    def get_inputs(self,init_inventory, eod_inventory, todays_order, item_number):
        item = eod_inventory.iloc[item_number]
        intial_quantity = init_inventory.query(f"product_id == '{item['product_id']}' and warehouse_id == '{item['warehouse_id']}'").iloc[0]['quantity']
        ordered_today = todays_order.query(f"product_id == '{item['product_id']}'") # TODO: Order history doesn't contain warehouse info
        print(f'Ordered Today {len(ordered_today)}')
        return {
            'warehouse_id': str(item['warehouse_id']),
            'intial_quantity': str(intial_quantity),
            'sold_quantity': str(len(ordered_today)),
            'product_id': str(item['product_id']),
            'remaining_quantity': str(item['quantity'])
        }

    def replenish(self, context: Dict) -> pd.DataFrame:

        """
        Returns the additional inventory to be procured for the next day
        """
        replenishment_orders = []

        total_products = len(context['eod_inventory_df'])
        crew = Crew(
            name="Warehouse Management Crew",
            process=Process.sequential,
            agents=[self.agent],
            tasks=[self.task],
            verbose=True
        )

        for item_number in range(total_products):
            try:
                # Get inputs for current product
                inputs = self.get_inputs(
                    context['init_inventory_df'],
                    context['eod_inventory_df'],
                    context['todays_order'],
                    item_number
                )
                # Get replenishment recommendation
                answer = crew.kickoff(inputs=inputs)
                predicted_quantity = answer.pydantic.quantity
                # Validate response
                if not isinstance(predicted_quantity, (int, float)):
                    print(f'Received unexpected response for product {inputs["product_id"]}: type {type(answer)} Answer {answer}')
                    print('response:\n', answer)
                    continue
                print("SPECIFIC Inputs ",inputs['warehouse_id'])
                print("SPECIFIC Inputs ",inputs['product_id'])
                replenishment_orders.append({
                    'warehouse_id': inputs['warehouse_id'],
                    'product_id': inputs['product_id'],
                    'quantity': predicted_quantity
                    })
                print(f'Processed product {item_number + 1}/{total_products}: {inputs["product_id"]}')

            except Exception as e:
                print(f'Error processing product {item_number}: {str(e)}')
                continue

        replenish_df = pd.DataFrame(replenishment_orders)
        print('\nAll replenishment orders:')
        print(replenish_df)
        return replenish_df

In [19]:
# Order Fulfillment Service

FULFILLMENT_VELOCITY = 1

class OrderFulfillmentService:
    def __init__(
        self,
        warehouses: pd.DataFrame,
        inventory: pd.DataFrame,
    ):
        self.warehouses = warehouses
        self.inventory = inventory

    def get_all_inventory(self, product_id: str, required_quantity: int) -> pd.DataFrame | OutOfStockError:
        # Get the inventory for the product with required quantity
        product_availability = self.inventory[
            (self.inventory["product_id"] == product_id)
            & (self.inventory["quantity"] >= required_quantity)
        ]
        if product_availability.empty:
            raise OutOfStockError(product_id)
        return product_availability

    def get_nearest_inventory(self, delivery_pincode: int,
                              available_inventory: pd.DataFrame) -> Dict:
        available_inventory = available_inventory.merge(
            self.warehouses, left_on="warehouse_id", right_on="warehouse_id"
        )
        available_inventory["distance"] = available_inventory["pincode"].apply(
            lambda warehouse_pincode: abs(warehouse_pincode - delivery_pincode)
        )
        return available_inventory.sort_values(by="distance", ascending=True).head(1).squeeze().to_dict()

    def update_inventory(
        self, selected_inventory: Dict, quantity: int
    ) -> None:
        self.inventory.loc[
            (self.inventory["warehouse_id"] == selected_inventory.get("warehouse_id"))
            & (self.inventory["product_id"] == selected_inventory.get("product_id")),
            "quantity",
        ] -= quantity

    def fulfill_order(
        self, product_id: str, required_quantity: int, delivery_pincode: int
    ) -> str:
        available_inventory = self.get_all_inventory(product_id, required_quantity)
        selected_inventory = self.get_nearest_inventory(delivery_pincode, available_inventory)
        self.update_inventory(selected_inventory, required_quantity)
        return selected_inventory

    def process(self, orders: pd.DataFrame) -> pd.DataFrame | OutOfStockError:
        results = []
        for _, order in orders.iterrows():
            selected_inventory = self.fulfill_order(order.product_id, order.quantity, order.delivery_pincode)
            results.append(
                {
                    "order_id": order.order_id,
                    "date": order.date,
                    "warehouse_id": selected_inventory.get("warehouse_id"),
                    "product_id": order.product_id,
                    "quantity": order.quantity,
                    "eta": selected_inventory.get("distance") * FULFILLMENT_VELOCITY,
                    "status": "Fulfilled",
                }
            )
        return pd.DataFrame(results)

# Driver

In [20]:
agent_tools = {
    "historic_sales_report_summary": HistoricSalesReportSummaryRetriever(),
    "warehouse_surge_capacity_increase_requester": WarehouseSurgeCapacityIncreaseRequester(),
}

---

# Code beyond this point should not be changed

In [21]:
def update_inventory_stock(eod_inventory: pd.DataFrame, replenish_inventory: pd.DataFrame):
    # takes current inventory and replenished inventory and updates the warehouse inventory per each product
    eod_inventory = eod_inventory.set_index(["product_id", "warehouse_id"])
    replenish_inventory = replenish_inventory.set_index(
        ["product_id", "warehouse_id"]
    )
    eod_inventory.update(replenish_inventory)
    return eod_inventory.reset_index()

In [22]:
def main(agent_tools: Dict[str, object]):
    print(INPUT_PATH)
    # load data
    warehouses = pd.read_csv(INPUT_PATH / "warehouse.csv")
    inventory = pd.read_csv(INPUT_PATH / "inventory.csv")
    orders = pd.read_csv(INPUT_PATH / "orders.csv")

    # create agent
    agent = InventoryReplenishmentService(agent_tools)

    # Create final df
    final_fulfillment = pd.DataFrame(columns=["order_id", "eta" ])

    # each iteration processes orders for a particular date
    for date, todays_order in  orders.groupby("date", sort=True):
        print(f"Processing orders for {date}")
        try:
            ofs = OrderFulfillmentService(warehouses, inventory.copy())
            fulfillments = ofs.process(todays_order)
            eod_inventory = ofs.inventory
            final_fulfillment = pd.concat([final_fulfillment, fulfillments[["order_id", "eta"]]], ignore_index=True)
        except OutOfStockError as error:
            print(error, "\nTerminating order processing")
            return
        print(f"Successfully fulfilled orders for {date}")

        # invoke agent to replenish the inventory
        context = {
            "init_inventory_df": inventory,
            "eod_inventory_df": eod_inventory,
            "warehouses": warehouses,
            "todays_order": todays_order,
        }
        print("Invoking Agent to replenish inventory for next day")
        # Print Context
        print_context(context)
        replenish_inventory = agent.replenish(context)

        # update the inventory
        inventory = update_inventory_stock(eod_inventory, replenish_inventory)
        print("Completed restocking inventory\n")

    try:
        final_fulfillment.to_csv(get_fullfillments_path(), encoding='utf-8', index=False)
        print("Success: File saved successfully to ",get_fullfillments_path())
    except Exception as e:
        print(f"Error: {e}")

In [23]:
def get_fullfillments_path():
    fulfillments_dir = os.path.join(WORKING_PATH, "Fulfillments")
    os.makedirs(fulfillments_dir, exist_ok=True)
    return os.path.join(fulfillments_dir, "final_fulfillments_.csv")

In [24]:
def print_context(context):
    pass
    # for name, df in context.items():
    #     print(f"### {name} ###")
    #     print(f"Length: {len(df)}")
    #     if len(df) > 10:  # Check if the dataframe has more than 10 rows
    #         # Display the first 5 and last 5 rows with "..."
    #         print(df.head(5).to_string(index=False))
    #         print("...")
    #         print(df.tail(5).to_string(index=False))
    #     else:
    #         # Display the entire dataframe
    #         print(df.to_string(index=False))
    #     print("\n" + "-" * 50 + "\n")


## Context

### 1. **Initial Inventory (`init_inventory_df`)**
   - `warehouse_id`, `product_id`, `quantity`

### 2. **End of Day Inventory (`eod_inventory_df`)**
   - `warehouse_id`, `product_id`, `quantity`

### 3. **Warehouses (`warehouses`)**
   - `warehouse_id`, `address`, `pincode`, `insured_value`

### 4. **Today's Orders (`todays_order`)**
   - `order_id`, `date`, `delivery_pincode`, `product_id`, `quantity`

In [25]:
main(agent_tools)

/kaggle/input/rts-phase-one-final
Processing orders for 2024-10-01
Successfully fulfilled orders for 2024-10-01
Invoking Agent to replenish inventory for next day


17:44:55 - LiteLLM:INFO: utils.py:2894 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai


Ordered Today 1
# Agent: Warehouse Inventory Planner
## Task: Given that 1 units of product PQQPSUQRU6 were sold today, and 100 units are remaining, choose the quantity to order so that sufficient quantity is available for tomorrow's operations in the warehouse


17:44:56 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
17:44:56 - LiteLLM:INFO: utils.py:2894 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Warehouse Inventory Planner
## Final Answer: 
{ "quantity": 1 }


SPECIFIC Inputs  CHE0055
SPECIFIC Inputs  PQQPSUQRU6
Processed product 1/37: PQQPSUQRU6
Ordered Today 2
# Agent: Warehouse Inventory Planner
## Task: Given that 2 units of product PESIRQ8UQT were sold today, and 10 units are remaining, choose the quantity to order so that sufficient quantity is available for tomorrow's operations in the warehouse


...



# Agent: Warehouse Inventory Planner
## Final Answer: 
{
  "quantity": 10
}


SPECIFIC Inputs  KOL0042
SPECIFIC Inputs  K55XZQEI1E
Processed product 35/37: K55XZQEI1E
Ordered Today 0
# Agent: Warehouse Inventory Planner
## Task: Given that 0 units of product JQXL6MWJBM were sold today, and 0 units are remaining, choose the quantity to order so that sufficient quantity is available for tomorrow's operations in the warehouse


17:51:31 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
17:51:31 - LiteLLM:INFO: utils.py:2894 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai




# Agent: Warehouse Inventory Planner
## Final Answer: 
{ "quantity": 10 }


SPECIFIC Inputs  KOL0042
SPECIFIC Inputs  JQXL6MWJBM
Processed product 36/37: JQXL6MWJBM
Ordered Today 1
# Agent: Warehouse Inventory Planner
## Task: Given that 1 units of product O1WNXFYBN4 were sold today, and 10 units are remaining, choose the quantity to order so that sufficient quantity is available for tomorrow's operations in the warehouse


17:51:32 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler




# Agent: Warehouse Inventory Planner
## Final Answer: 
{ "quantity": 5 }


SPECIFIC Inputs  KOL0042
SPECIFIC Inputs  O1WNXFYBN4
Processed product 37/37: O1WNXFYBN4

All replenishment orders:
   warehouse_id  product_id  quantity
0       CHE0055  PQQPSUQRU6        10
1       CHE0055  PESIRQ8UQT         2
2       CHE0055  PY0DC8LGG1         5
3       KOL0031  PY0DC8LGG1         5
4       KOL0031  PYG3WEVWGO       100
5       KOL0031  PESIRQ8UQT         6
6       DEL0009  PYG3WEVWGO         5
7       DEL0009  PESIRQ8UQT       100
8       DEL0009  PQQPSUQRU6         5
9       MUM0075  PY0DC8LGG1        10
10      MUM0075  PESIRQ8UQT        10
11      MUM0075  PQQPSUQRU6        10
12      BEN0077  PY0DC8LGG1         0
13      BEN0077  PQQPSUQRU6         4
14      BEN0077  PYG3WEVWGO        10
15      KOL0063  PESIRQ8UQT        10
16      KOL0063  PY0DC8LGG1         6
17      KOL0063  PQQPSUQRU6         5
18      MUM0072  PQQPSUQRU6        10
19      MUM0072  PESIRQ8UQT         5
20      M